In [1]:
import tensorflow as tf
from tensorflow import keras
import kerastuner
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os
import json
import datetime

plt.rcParams["figure.figsize"] = (20, 5)

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [2]:
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
'''
log_dir = "logs/fit/" + timestamp
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
'''
version_dir = "version/" + timestamp 

os.makedirs(version_dir)
timestamp

'20200905-103143'

In [3]:
dataset_name = "SEG_2"

In [4]:
dataset = np.genfromtxt("data/{}_train_set.csv".format(dataset_name), delimiter="\n", dtype=np.float32) #np.int64
dataset

array([0., 0., 0., ..., 1., 3., 1.], dtype=float32)

In [5]:
word_index = np.genfromtxt("data/word_index.csv", delimiter="\n", dtype=np.int64)
vocab_size = len(word_index)
vocab_size

14882

In [6]:
param_list = dict()

param_list["PAST_HISTORY"] = 16
param_list["FUTURE_TARGET"] = 8
param_list["BATCH_SIZE"] = 128
param_list["EPOCHS"] = 100
param_list["BUFFER_SIZE"] = 200000
param_list["VOCAB_SIZE"] = vocab_size
param_list["LEARNING_RATE"] = 0.01
param_list["NUM_1_NEURONS"] = 177
param_list["NUM_2_NEURONS"] = 177
param_list["DROPOUT_1"] = 0.1
param_list["DROPOUT_2"] = 0.2

In [7]:
def generate_timeseries(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, n_feature)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        #data.append(dataset[indices])
        labels.append(np.reshape(dataset[i:i+target_size], (target_size, 1)))
        #labels.append(dataset[i:i+target_size])
    return np.array(data), np.array(labels)

In [8]:
x_train, y_train = generate_timeseries(dataset, 0, None, param_list["PAST_HISTORY"], param_list["FUTURE_TARGET"])
x_train.shape, y_train.shape

((14858, 16, 1), (14858, 8, 1))

In [9]:
x_train[10], y_train[10]

(array([[  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [933.],
        [  0.],
        [  0.],
        [  0.],
        [  0.]], dtype=float32),
 array([[ 0.],
        [ 0.],
        [48.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.]], dtype=float32))

In [10]:
model = keras.models.Sequential()
model.add(keras.layers.Bidirectional(tf.keras.layers.LSTM(param_list["NUM_1_NEURONS"])))
model.add(keras.layers.Dropout(param_list["DROPOUT_1"]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.RepeatVector(param_list["FUTURE_TARGET"]))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(param_list["NUM_2_NEURONS"], return_sequences=True)))
model.add(keras.layers.Dropout(param_list["DROPOUT_2"]))
model.add(keras.layers.BatchNormalization())
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(param_list["NUM_2_NEURONS"], return_sequences=True)))
model.add(keras.layers.Dropout(param_list["DROPOUT_2"]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.TimeDistributed(keras.layers.Dense(param_list["VOCAB_SIZE"], activation='softmax')))
model.compile(optimizer=keras.optimizers.Adam(param_list["LEARNING_RATE"]), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
model_history = model.fit(x_train, y_train, batch_size=param_list["BATCH_SIZE"], validation_split=0.2, epochs=param_list["EPOCHS"])

0s - loss: 0.4909 - accuracy: 0.8787/93 [===========================>..] - ETA: 0s - loss: 0.4902 - accuracy: 0.8789/93 [===========================>..] - ETA: 0s - loss: 0.4917 - accuracy: 0.8791/93 [============================>.] - ETA: 0s - loss: 0.4899 - accuracy: 0.8793/93 [==============================] - ETA: 0s - loss: 0.4912 - accuracy: 0.8793/93 [==============================] - 4s 43ms/step - loss: 0.4912 - accuracy: 0.8776 - val_loss: 0.5033 - val_accuracy: 0.8273
Epoch 96/100
 1/93 [..............................] - ETA: 0s - loss: 0.4941 - accuracy: 0.88 3/93 [..............................] - ETA: 2s - loss: 0.4746 - accuracy: 0.88 5/93 [>.............................] - ETA: 2s - loss: 0.4556 - accuracy: 0.88 7/93 [=>............................] - ETA: 2s - loss: 0.4791 - accuracy: 0.88 9/93 [=>............................] - ETA: 2s - loss: 0.4593 - accuracy: 0.8811/93 [==>...........................] - ETA: 2s - loss: 0.4521 - accuracy: 0.8813/93 [===>............

In [12]:
model.save("version/{}/model.h5".format(timestamp))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #
bidirectional (Bidirectional multiple                  253464
_________________________________________________________________
dropout (Dropout)            multiple                  0
_________________________________________________________________
batch_normalization (BatchNo multiple                  1416
_________________________________________________________________
repeat_vector (RepeatVector) multiple                  0
_________________________________________________________________
bidirectional_1 (Bidirection multiple                  753312
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0
_________________________________________________________________
batch_normalization_1 (Batch multiple                  1416
________________________________________